In [1]:
import cv2
import numpy as np

from glob import glob
import os
from os.path import join
import pandas as pd

In [2]:
CORREL_ACC = 0.85
CHISQR_ACC = 2500
INTERSECT_ACC = 600
BHATTACHARYYA_ACC = 0.3

In [3]:
def list_images():
    cwd = os.getcwd()
    
    files = []
    for ext in ('*.jpg', '*.png'):
        #files.extend(glob(join(cwd, ext)))
        files.extend(glob(ext))
    
    return files


In [4]:
def get_class(parse):
    str_list = []

    str_list = parse.split('1')
    str_list = str_list[0].split('2')

    return str_list[0]

In [5]:
def verify_hit(class_1, class_2, value, acceptance, higher):
    if (higher == True):
        if ( (value >= acceptance) and (class_1 == class_2) ):
            return 1
        else:
            return 0
    elif (higher == False):
        if ( (value <= acceptance) and (class_1 == class_2) ):
            return 1
        else:
            return 0

In [6]:
files = list_images()
files

['hulk1.png',
 'hulk2.png',
 'iron1.png',
 'iron2.png',
 'k3po1.png',
 'k3po2.png',
 'magneto1.png',
 'magneto2.png',
 'trooper1.png',
 'trooper2.png',
 'vader1.png',
 'vader2.png',
 'volve1.png',
 'volve2.png']

In [ ]:
img1 = cv2.imread(files[0])
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)

img2 = cv2.imread(files[1])
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

result = []
hist1 = cv2.calcHist([img1], [0,1,2], None, [8,8,8], [0, 256, 0, 256, 0, 256])
hist2 = cv2.calcHist([img2], [0,1,2], None, [8,8,8], [0, 256, 0, 256, 0, 256])
cv2.normalize(hist1, hist1, 0, 255, cv2.NORM_MINMAX)
cv2.normalize(hist2, hist2, 0, 255, cv2.NORM_MINMAX)
sc = cv2.compareHist(hist1, hist2, cv2.HISTCMP_BHATTACHARYYA)
print(sc)


In [ ]:
img1 = cv2.imread(files[0])
img2 = cv2.imread(files[1])
result = []
for ch, col in enumerate(['b', 'g', 'r']):
    hist1 = cv2.calcHist([img1], [ch], None, [256], [0,255])
    hist2 = cv2.calcHist([img2], [ch], None, [256], [0,255])
    cv2.normalize(hist1, hist1, 0, 255, cv2.NORM_MINMAX)
    cv2.normalize(hist2, hist2, 0, 255, cv2.NORM_MINMAX)
    sc = cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)
    result.append(sc)
print(max(result))


In [ ]:
img1 = cv2.imread(files[0],0)
img2 = cv2.imread(files[0],0)

for ch, col in enumerate(['b', 'g', 'r']):
    hist1 = cv2.calcHist([img1], [0], None, [256], [0,255])
    hist2 = cv2.calcHist([img2], [0], None, [256], [0,255])
    cv2.normalize(hist1, hist1, 0, 255, cv2.NORM_MINMAX)
    cv2.normalize(hist2, hist2, 0, 255, cv2.NORM_MINMAX)
    sc = cv2.compareHist(hist1, hist2, cv2.HISTCMP_BHATTACHARYYA)
    print(sc)

In [ ]:
# cv2.HISTCMP_CORREL
# cv2.HISTCMP_CHISQR
# cv2.HISTCMP_INTERSECT
# cv2.HISTCMP_BHATTACHARYYA

# CORREL_ACC = 0.8
# CHISQR_ACC = 3000
# INTERSECT_ACC = 500
# BHATTACHARYYA_ACC = 0.3

In [16]:
# IMAGENS COLORIDAS

df = pd.DataFrame()
for i in range(0,len(files),1):
    temporary = []
    
    selected_image = cv2.imread(files[i])
    selected_image = cv2.cvtColor(selected_image, cv2.COLOR_BGR2RGB)
    selected_image_class = get_class(files[i])
    

    CORREL_hit = 0
    CHISQR_hit = 0
    INTERSECT_hit = 0
    BHATTACHARYYA_hit = 0
    for f in range(0,len(files),1):
        if (i != f):
            print(files[i], files[f])
            compared_image = cv2.imread(files[f])
            compared_image = cv2.cvtColor(compared_image, cv2.COLOR_BGR2RGB)
            compared_image_class = get_class(files[f])
            
            hist1 = cv2.calcHist([selected_image], [0,1,2], None, [8,8,8], [0,256,0,256,0,256])
            hist2 = cv2.calcHist([compared_image], [0,1,2], None, [8,8,8], [0,256,0,256,0,256])
            cv2.normalize(hist1, hist1, 0, 255, cv2.NORM_MINMAX)
            cv2.normalize(hist2, hist2, 0, 255, cv2.NORM_MINMAX)


            CORREL = cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)
            CHISQR = cv2.compareHist(hist1, hist2, cv2.HISTCMP_CHISQR)
            INTERSECT = cv2.compareHist(hist1, hist2, cv2.HISTCMP_INTERSECT)
            BHATTACHARYYA = cv2.compareHist(hist1, hist2, cv2.HISTCMP_BHATTACHARYYA)
            

            CORREL_hit += verify_hit( class_1 = selected_image_class, class_2 = compared_image_class, value = CORREL, acceptance = CORREL_ACC, higher = True)
            CHISQR_hit += verify_hit( class_1 = selected_image_class, class_2 = compared_image_class, value = CHISQR, acceptance = CHISQR_ACC, higher = False)
            INTERSECT_hit += verify_hit( class_1 = selected_image_class, class_2 = compared_image_class, value = INTERSECT, acceptance = INTERSECT_ACC, higher = True)
            BHATTACHARYYA_hit += verify_hit( class_1 = selected_image_class, class_2 = compared_image_class, value = BHATTACHARYYA, acceptance = BHATTACHARYYA_ACC, higher = False)

            # print("CORREL = ", CORREL)
            print("CHISQR = ", CHISQR)
            # print("INTERSECT = ", INTERSECT)
            # print("BHATTACHARYYA = ", BHATTACHARYYA)
            # print("\n")


    temporary.append(files[i])
    temporary.append(CORREL_hit)
    temporary.append(CHISQR_hit)
    temporary.append(INTERSECT_hit)
    temporary.append(BHATTACHARYYA_hit)
    # print(temporary)

    temp_df = pd.DataFrame(temporary).transpose()
    df = pd.concat([df, temp_df], axis=0, ignore_index=True)


df = df.rename(columns=({0:'IMAGE NAME',
                    1:'CORREL hit',
                    2:'CHISQR hit',
                    3:'INTERSECT hit',
                    4:'BHATTACHARYYA hit'}))

sum_df = pd.DataFrame( [[ 'TOTAL GERAL', sum(df['CORREL hit']), sum(df['CHISQR hit']), sum(df['INTERSECT hit']), sum(df['BHATTACHARYYA hit']) ]], columns=['IMAGE NAME', 'CORREL hit', 'CHISQR hit', 'INTERSECT hit', 'BHATTACHARYYA hit'] )
df = pd.concat([df, sum_df], axis=0, ignore_index=True)

df

hulk1.png hulk2.png
CHISQR =  4183.6781674384765
hulk1.png iron1.png
CHISQR =  9313.780000455312
hulk1.png iron2.png
CHISQR =  12116.765262071174
hulk1.png k3po1.png
CHISQR =  835.0193602708364
hulk1.png k3po2.png
CHISQR =  622.3945422405407
hulk1.png magneto1.png
CHISQR =  3482.1329280865584
hulk1.png magneto2.png
CHISQR =  3425.1551671704437
hulk1.png trooper1.png
CHISQR =  3576.6630659629814
hulk1.png trooper2.png
CHISQR =  15319.90152053084
hulk1.png vader1.png
CHISQR =  7127.7716925049335
hulk1.png vader2.png
CHISQR =  9204.516568979403
hulk1.png volve1.png
CHISQR =  1443.862618133968
hulk1.png volve2.png
CHISQR =  1318.7586706234108
hulk2.png hulk1.png
CHISQR =  2291.213140655601
hulk2.png iron1.png
CHISQR =  10550.364278629604
hulk2.png iron2.png
CHISQR =  12382.29174411498
hulk2.png k3po1.png
CHISQR =  1344.725916654367
hulk2.png k3po2.png
CHISQR =  1133.7944239661383
hulk2.png magneto1.png
CHISQR =  26772.31114918499
hulk2.png magneto2.png
CHISQR =  41855.094403755975
hulk2.pn

,IMAGE NAME,CORREL hit,CHISQR hit,INTERSECT hit,BHATTACHARYYA hit
0,hulk1.png,1,0,1,1
1,hulk2.png,1,1,1,1
2,iron1.png,1,1,1,1
3,iron2.png,1,1,1,1
4,k3po1.png,1,1,0,1
5,k3po2.png,1,1,0,1
6,magneto1.png,1,1,1,1
7,magneto2.png,1,1,1,1
8,trooper1.png,0,1,1,1
9,trooper2.png,0,1,1,1


In [8]:
# IMAGENS EM PRETO E BRANCO

df = pd.DataFrame()
for i in range(0,len(files),1):
    temporary = []
    
    selected_image = cv2.imread(files[i],0)
    selected_image_class = get_class(files[i])
    

    CORREL_hit = 0
    CHISQR_hit = 0
    INTERSECT_hit = 0
    BHATTACHARYYA_hit = 0
    for f in range(0,len(files),1):
        if (i != f):
            # print(files[i], files[f])
            compared_image = cv2.imread(files[f],0)
            compared_image_class = get_class(files[f])
            
            hist1 = cv2.calcHist([selected_image], [0], None, [256], [0,256])
            hist2 = cv2.calcHist([compared_image], [0], None, [256], [0,256])
            cv2.normalize(hist1, hist1, 0, 255, cv2.NORM_MINMAX)
            cv2.normalize(hist2, hist2, 0, 255, cv2.NORM_MINMAX)


            CORREL = cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)
            CHISQR = cv2.compareHist(hist1, hist2, cv2.HISTCMP_CHISQR)
            INTERSECT = cv2.compareHist(hist1, hist2, cv2.HISTCMP_INTERSECT)
            BHATTACHARYYA = cv2.compareHist(hist1, hist2, cv2.HISTCMP_BHATTACHARYYA)
            

            CORREL_hit += verify_hit( class_1 = selected_image_class, class_2 = compared_image_class, value = CORREL, acceptance = CORREL_ACC, higher = True)
            CHISQR_hit += verify_hit( class_1 = selected_image_class, class_2 = compared_image_class, value = CHISQR, acceptance = CHISQR_ACC, higher = False)
            INTERSECT_hit += verify_hit( class_1 = selected_image_class, class_2 = compared_image_class, value = INTERSECT, acceptance = INTERSECT_ACC, higher = True)
            BHATTACHARYYA_hit += verify_hit( class_1 = selected_image_class, class_2 = compared_image_class, value = BHATTACHARYYA, acceptance = BHATTACHARYYA_ACC, higher = False)

            # print("CORREL = ", CORREL)
            # print("CHISQR = ", CHISQR)
            # print("INTERSECT = ", INTERSECT)
            # print("BHATTACHARYYA = ", BHATTACHARYYA)
            # print("\n")


    temporary.append(files[i])
    temporary.append(CORREL_hit)
    temporary.append(CHISQR_hit)
    temporary.append(INTERSECT_hit)
    temporary.append(BHATTACHARYYA_hit)
    # print(temporary)

    temp_df = pd.DataFrame(temporary).transpose()
    df = pd.concat([df, temp_df], axis=0, ignore_index=True)


df = df.rename(columns=({0:'IMAGE NAME',
                    1:'CORREL hit',
                    2:'CHISQR hit',
                    3:'INTERSECT hit',
                    4:'BHATTACHARYYA hit'}))

sum_df = pd.DataFrame( [[ 'TOTAL GERAL', sum(df['CORREL hit']), sum(df['CHISQR hit']), sum(df['INTERSECT hit']), sum(df['BHATTACHARYYA hit']) ]], columns=['IMAGE NAME', 'CORREL hit', 'CHISQR hit', 'INTERSECT hit', 'BHATTACHARYYA hit'] )
df = pd.concat([df, sum_df], axis=0, ignore_index=True)
df

,IMAGE NAME,CORREL hit,CHISQR hit,INTERSECT hit,BHATTACHARYYA hit
0,hulk1.png,0,0,1,1
1,hulk2.png,0,0,1,1
2,iron1.png,0,0,1,1
3,iron2.png,0,0,1,1
4,k3po1.png,1,1,1,1
5,k3po2.png,1,1,1,1
6,magneto1.png,1,1,1,1
7,magneto2.png,1,0,1,1
8,trooper1.png,0,0,1,1
9,trooper2.png,0,0,1,1


In [ ]:
for i in range(0,1,1):
    
    for f in range(0,len(files),1):
        if (i != f):
            print(i,f)

In [ ]:
testes = pd.DataFrame([], columns=['EXAMES DO MÉTODO RT-PCR (por dia)', 'EXAMES DO MÉTODO RT-PCR (por semana)', 'One Step Test', 'Testes de Antígenos (realizados nos últimos 7 dias)'])
testes

In [17]:
d={"a": 3, "b": 2}
minimo = min(d, key=d.get)
print(minimo)
maximo = max(d, key=d.get)
print(maximo)

b
a


In [30]:
# IMAGENS COLORIDAS

df = pd.DataFrame()
for i in range(0,len(files),1):
    temporary = []
    dic_correl = {}
    dic_chisqr = {}
    dic_intersect = {}
    dic_bhattacharyya = {}
    
    selected_image = cv2.imread(files[i])
    selected_image = cv2.cvtColor(selected_image, cv2.COLOR_BGR2RGB)
    selected_image_class = get_class(files[i])
    

    CORREL_hit, correl_max, CHISQR_hit, chisqr_min, INTERSECT_hit, intersect_max, BHATTACHARYYA_hit, bhattacharyya_min = 0, 0, 0, 0, 0, 0, 0, 0
    for f in range(0,len(files),1):
        if (i != f):
            # print(files[i], files[f])
            compared_image = cv2.imread(files[f])
            compared_image = cv2.cvtColor(compared_image, cv2.COLOR_BGR2RGB)
            
            hist1 = cv2.calcHist([selected_image], [0,1,2], None, [8,8,8], [0,256,0,256,0,256])
            hist2 = cv2.calcHist([compared_image], [0,1,2], None, [8,8,8], [0,256,0,256,0,256])
            cv2.normalize(hist1, hist1, 0, 255, cv2.NORM_MINMAX)
            cv2.normalize(hist2, hist2, 0, 255, cv2.NORM_MINMAX)


            CORREL = cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)
            CHISQR = cv2.compareHist(hist1, hist2, cv2.HISTCMP_CHISQR)
            INTERSECT = cv2.compareHist(hist1, hist2, cv2.HISTCMP_INTERSECT)
            BHATTACHARYYA = cv2.compareHist(hist1, hist2, cv2.HISTCMP_BHATTACHARYYA)


            # print("CORREL = ", CORREL)
            # print("CHISQR = ", CHISQR)
            # print("INTERSECT = ", INTERSECT)
            # print("BHATTACHARYYA = ", BHATTACHARYYA)
            # print("\n")
            

            dic_correl[files[f]] = CORREL
            dic_chisqr[files[f]] = CHISQR
            dic_intersect[files[f]] = INTERSECT
            dic_bhattacharyya[files[f]] = BHATTACHARYYA

    
    
    correl_max = max(dic_correl, key=dic_correl.get)
    CORREL_hit = verify( class_1 = selected_image_class, class_2 = get_class(correl_max))


    chisqr_min = min(dic_chisqr, key=dic_chisqr.get)
    CHISQR_hit = verify( class_1 = selected_image_class, class_2 = get_class(chisqr_min))


    intersect_max = max(dic_intersect, key=dic_intersect.get)
    INTERSECT_hit = verify( class_1 = selected_image_class, class_2 = get_class(intersect_max))


    bhattacharyya_min = min(dic_bhattacharyya, key=dic_bhattacharyya.get)
    BHATTACHARYYA_hit = verify( class_1 = selected_image_class, class_2 = get_class(bhattacharyya_min))


    temporary.append(files[i])
    temporary.append(CORREL_hit)
    temporary.append(CHISQR_hit)
    temporary.append(INTERSECT_hit)
    temporary.append(BHATTACHARYYA_hit)
    # print(temporary)

    temp_df = pd.DataFrame(temporary).transpose()
    df = pd.concat([df, temp_df], axis=0, ignore_index=True)


df = df.rename(columns=({0:'IMAGE NAME',
                    1:'CORREL hit',
                    2:'CHISQR hit',
                    3:'INTERSECT hit',
                    4:'BHATTACHARYYA hit'}))

sum_df = pd.DataFrame( [[ 'TOTAL GERAL', sum(df['CORREL hit']), sum(df['CHISQR hit']), sum(df['INTERSECT hit']), sum(df['BHATTACHARYYA hit']) ]], columns=['IMAGE NAME', 'CORREL hit', 'CHISQR hit', 'INTERSECT hit', 'BHATTACHARYYA hit'] )
df = pd.concat([df, sum_df], axis=0, ignore_index=True)

df

,IMAGE NAME,CORREL hit,CHISQR hit,INTERSECT hit,BHATTACHARYYA hit
0,hulk1.png,1,0,1,1
1,hulk2.png,0,0,0,1
2,iron1.png,0,0,1,1
3,iron2.png,1,1,0,1
4,k3po1.png,1,1,0,1
5,k3po2.png,1,1,0,1
6,magneto1.png,1,1,1,1
7,magneto2.png,1,1,1,1
8,trooper1.png,0,1,1,1
9,trooper2.png,0,1,0,1


In [24]:
print(dic_correl)
minimo = min(dic_correl, key=dic_correl.get)
print(minimo)

{'hulk2.png': 0.8794585939060866, 'iron1.png': 0.8236188578126534, 'iron2.png': 0.84404566687619, 'k3po1.png': 0.8538218350439268, 'k3po2.png': 0.8576476521614392, 'magneto1.png': 0.8031793918993911, 'magneto2.png': 0.7936642532445187, 'trooper1.png': 0.4560188229789735, 'trooper2.png': 0.6802362543604153, 'vader1.png': 0.037810696314975946, 'vader2.png': 0.36345940307301383, 'volve1.png': 0.7678838964368921, 'volve2.png': 0.8410970119242422}
vader1.png


In [27]:
def verify (class_1, class_2):
    if (class_1 == class_2):
        return 1
    else:
        return 0

In [29]:
compared_image_class = get_class(bhattacharyya_min)
print(compared_image_class)

volve
